# Which activation function to use?
#### Author: JP Melo

### Imports

In [1]:
from derpinns.nn import *
from derpinns.utils import *
from derpinns.trainer import *
import torch.nn as nn
import torch

## Parameters

In [2]:
# Fix seed for reproducibility
torch.manual_seed(0)
np.random.seed(0)

# Global parameters
assets = 3

sampler = "pseudo"
nn_shape = "10x3"
device = torch.device("cpu")
dtype = torch.float32

# Define option valuation params
params = OptionParameters(
    n_assets=assets,
    tau=1.0,
    sigma=np.array([0.2] * assets),
    rho=np.eye(assets) + 0.25 * (np.ones((assets, assets)) - np.eye(assets)),
    r=0.05,
    strike=100,
    payoff=payoff
)

boundary_samples = 1_000
interior_samples = boundary_samples*assets*2
initial_samples = boundary_samples*assets*2

## Training

### Tanh

In [ ]:
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32,
    activation=nn.Tanh(),
).apply(weights_init).to(device)

model.train()

dataset = SampledDataset(
    params, interior_samples, initial_samples, boundary_samples, sampler, dtype, device, seed=0)

optimizer = SSBroyden(
    model.parameters(),
    max_iter=500,
)

batch_size = len(dataset)  # we use all samples

closure = DimlessBS()\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)

trainer.train()

SSBroyden training:   9%|▉         | 47/500 [01:03<09:54,  1.31s/it, Interior=0.007814, Boundary=0.029259, Initial=0.089855, Total=0.126927, Max Error=76.897736, L2 Error=0.123560] 

In [ ]:
with_tanh = trainer.closure.get_state()
plot_loss(with_tanh, smooth=True, smooth_window=50)

tanh_l2 = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device, seed=42)['l2_rel_error']
print("L2 Error: ", tanh_l2*100)

### ReLU

In [ ]:
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32,
    activation=nn.ReLU(),
).apply(weights_init).to(device)

model.train()

dataset = SampledDataset(
    params, interior_samples, initial_samples, boundary_samples, sampler, dtype, device, seed=0)

optimizer = SSBroyden(
    model.parameters(),
    max_iter=500,
)

batch_size = len(dataset)  # we use all samples

closure = DimlessBS()\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)

trainer.train()

In [ ]:
with_relu = trainer.closure.get_state()
plot_loss(with_relu, smooth=True, smooth_window=50)

relu_l2 = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device, seed=42)['l2_rel_error']
print("L2 Error: ", relu_l2*100)

### SoftPlus

In [ ]:
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32,
    activation=nn.Softplus(),
).apply(weights_init).to(device)

model.train()

dataset = SampledDataset(
    params, interior_samples, initial_samples, boundary_samples, sampler, dtype, device, seed=0)

optimizer = SSBroyden(
    model.parameters(),
    max_iter=500,
)

batch_size = len(dataset)  # we use all samples

closure = DimlessBS()\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)

trainer.train()

In [ ]:
with_softplus = trainer.closure.get_state()
plot_loss(with_softplus, smooth=True, smooth_window=50)

softplus_l2 = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device, seed=42)['l2_rel_error']
print("L2 Error: ", softplus_l2*100)

### ELU

In [ ]:
model = build_nn(
    nn_shape=nn_shape,
    input_dim=assets,
    dtype=torch.float32,
    activation=nn.ELU(),
).apply(weights_init).to(device)

model.train()

dataset = SampledDataset(
    params, interior_samples, initial_samples, boundary_samples, sampler, dtype, device, seed=0)

optimizer = SSBroyden(
    model.parameters(),
    max_iter=500,
)

batch_size = len(dataset)  # we use all samples

closure = DimlessBS()\
    .with_dataset(dataset, loader_opts={'batch_size': batch_size, "shuffle": True, "pin_memory": True})\
    .with_model(model)\
    .with_device(device)\
    .with_dtype(dtype)

trainer = PINNTrainer()\
    .with_optimizer(optimizer)\
    .with_device(device)\
    .with_dtype(dtype)\
    .with_training_step(closure)

trainer.train()

In [ ]:
with_elu = trainer.closure.get_state()
plot_loss(with_elu, smooth=True, smooth_window=50)

elu_l2 = compare_with_mc(model, params, n_prices=200,
                          n_simulations=10_000, dtype=dtype, device=device, seed=42)['l2_rel_error']
print("L2 Error: ", elu_l2*100)

### Compare runs

In [ ]:
compare_error_histories(
    [with_tanh, with_relu, with_softplus, with_elu],
    labels=["Tanh", "ReLU", "Softplus", "ELU"],
    smooth=False,
)